In [1]:
import pandas as pd
import numpy as np
import env

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [2]:
query ='''select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
from propertylandusetype
	join properties_2017
		using (propertylandusetypeid)
WHERE propertylandusedesc = ("Single Family Residential")'''

In [3]:
 url = env.get_db_url("zillow")

In [4]:
zillow=pd.read_sql(query,url)

In [5]:
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [6]:
zillow.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [7]:
zillow.yearbuilt.isnull().value_counts()

yearbuilt
False    2143526
True        9337
Name: count, dtype: int64

In [8]:
(9337/2143526) * 100

0.43559070428816815

In [9]:
zillow.yearbuilt.isnull

<bound method Series.isnull of 0             NaN
1             NaN
2             NaN
3             NaN
4          2005.0
            ...  
2152858    2015.0
2152859    2014.0
2152860       NaN
2152861    2015.0
2152862    2014.0
Name: yearbuilt, Length: 2152863, dtype: float64>

In [10]:
zillow.yearbuilt.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2152863 entries, 0 to 2152862
Series name: yearbuilt
Non-Null Count    Dtype  
--------------    -----  
2143526 non-null  float64
dtypes: float64(1)
memory usage: 16.4 MB


In [11]:
zillow = zillow.dropna()

In [12]:
zillow.yearbuilt = zillow.yearbuilt.astype(int)

In [13]:
zillow.fips

4          6037.0
6          6037.0
7          6037.0
11         6037.0
14         6037.0
            ...  
2152856    6037.0
2152858    6059.0
2152859    6059.0
2152861    6059.0
2152862    6037.0
Name: fips, Length: 2140235, dtype: float64

In [14]:
zillow.fips = zillow.fips.astype(int)

In [15]:
zillow.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                         int64
taxamount                       float64
fips                              int64
dtype: object

In [16]:

zillow.rename(columns={"calculatedfinishedsquarefeet": "calfinsqft"}, inplace=True)


In [17]:
zillow.bedroomcnt = zillow.bedroomcnt.astype(int)
zillow.taxvaluedollarcnt = zillow.taxvaluedollarcnt.astype(int)
zillow.calfinsqft = zillow.calfinsqft.astype(int)

In [18]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   bedroomcnt         int64  
 1   bathroomcnt        float64
 2   calfinsqft         int64  
 3   taxvaluedollarcnt  int64  
 4   yearbuilt          int64  
 5   taxamount          float64
 6   fips               int64  
dtypes: float64(2), int64(5)
memory usage: 130.6 MB


In [19]:
import wrangle

In [20]:
zillow['taxamount_bins'] = pd.cut(zillow['taxamount'], bins=4)

In [21]:
from sklearn.model_selection import train_test_split

def wrangle_zillow(fin=zillow, target_column="taxamount_bins"):
    '''Takes the data frame and breaks it into train, test, and split information, with two splits'''

    # Use the specified target column for stratification
    target = fin[target_column]

    train, validate_test = train_test_split(fin,
                     train_size=0.6,
                     random_state=123,
                     #stratify=target
                    )

    validate, test = train_test_split(validate_test,
                                     train_size=0.5,
                                     random_state=123,
                                     #stratify=validate_test[target]
                                    )

    return train, validate, test


In [22]:
value_counts = zillow['taxamount'].value_counts() 
rare_classes = value_counts[value_counts == 1].index.tolist()

In [23]:
rare_classes

[6022.67,
 3891.81,
 6369.79,
 5805.99,
 4663.28,
 12682.37,
 7609.63,
 1404.23,
 12132.78,
 8785.13,
 15374.62,
 10008.86,
 14047.1,
 8100.33,
 6969.1,
 33541.92,
 4246.63,
 13032.38,
 6798.76,
 1202.05,
 4924.35,
 74057.07,
 7165.31,
 2124.07,
 28440.56,
 11641.76,
 5779.63,
 1883.93,
 16588.73,
 23101.61,
 36234.1,
 3933.09,
 25657.88,
 8340.98,
 8101.47,
 7067.79,
 4081.47,
 5557.41,
 7859.58,
 7310.43,
 14419.82,
 3167.21,
 7515.27,
 29617.66,
 7821.84,
 4691.8,
 6371.03,
 6964.59,
 14894.9,
 14908.13,
 3758.35,
 3984.41,
 646.99,
 2837.37,
 5807.48,
 32453.24,
 866.83,
 2318.63,
 18901.82,
 7224.21,
 9236.97,
 6513.59,
 8737.81,
 7916.6,
 8064.04,
 15108.88,
 6485.56,
 9544.54,
 8139.61,
 6600.72,
 546.35,
 7006.65,
 101760.3,
 9188.58,
 5487.92,
 8368.56,
 14364.42,
 647.31,
 13292.85,
 35234.18,
 14343.78,
 1565.23,
 3087.98,
 2904.39,
 5783.79,
 977.35,
 8372.02,
 9148.03,
 4119.18,
 14610.82,
 2793.87,
 7750.01,
 4778.61,
 19150.31,
 6385.86,
 9305.5,
 4830.82,
 7459.49,
 481

In [ ]:
zillow['taxamount'].replace(rare_classes, 'other_class')

In [ ]:
a,b,c= wrangle_zillow()

In [ ]:
a.head(1)

In [ ]:
print(zillow['taxamount'].value_counts())

In [ ]:
 zillow['taxamount'] = zillow["taxamount",bin=4]

In [ ]:
zillow

In [ ]:
a